In [3]:
!pip install numpy librosa
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import os
import numpy as np
import librosa
import torch
import torch.nn.functional as F

class ASVspoof19Loader:
    def __init__(self, audio_dir, label_file, sample_rate=16000):
        self.audio_dir   = audio_dir
        self.label_file  = label_file
        self.sample_rate = sample_rate
        self.label_map   = self.parsed_label_file()

    def parsed_label_file(self):
        label_dict = {}
        with open(self.label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                # ← use the 2nd column (parts[1]) for the actual audio filename
                file_id  = parts[1]
                filename = file_id + '.flac'
                label    = 0 if parts[-1]=='bonafide' else 1
                label_dict[filename] = label
        return label_dict

    def load_audio(self, filepath):
        try:
            wav, sr = librosa.load(filepath, sr=self.sample_rate)
            return wav / (np.max(np.abs(wav)) + 1e-9)
        except Exception as e:
            print(f"[Error] {filepath}: {e}")
            return None

    def load_dataset(self):
        data = []
        for fname, label in self.label_map.items():
            full_path = os.path.join(self.audio_dir, fname)
            if os.path.exists(full_path):
                audio = self.load_audio(full_path)
                if audio is not None:
                    data.append((audio, label))
            else:
                print(f"[Warning] Missing: {full_path}")
        return data

# ——— Test it ———
audio_folder  = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_train\flac"
protocol_file = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_cm_protocols\ASVspoof2019.LA.cm.train.trn.txt"

loader  = ASVspoof19Loader(audio_folder, protocol_file)
dataset = loader.load_dataset()

print("Total files loaded:", len(dataset))
if dataset:
    print("Sample waveform:", dataset[0][0][:10], "Label:", dataset[0][1])
else:
    print("No files loaded—check that your protocol and folder really match.")


In [ ]:
###OKR 3 and OKR4###
#Feature extraction
import librosa.display
import matplotlib.pyplot as plt

class FeatureExtractor:
    def __init__(self, dataset, sr = 16000):
        self.dataset = dataset
        self.sr = sr

    def extract_mfcc(self, y, n_mfcc = 13): #MFCC
        return librosa.feature.mfcc(y=y, sr = self.sr, n_mfcc = n_mfcc)
    
    def extract_lpc(self, y, order = 16): #LPC
        #using a 25ms window
        frame = y[:int(self.sr * 0.025)]
        return librosa.lpc(y=frame, order=order)
    
    def extract_cqt(self, y): #CQT
        return librosa.cqt(y=y, sr=self.sr)

    def extract_log_mel(self, y, n_mels=128): #preferring log mel since it is the standard for audio detection CNNs.
        mel = librosa.feature.melspectrogram(y=y, sr=self.sr, n_mels=n_mels)
        log_mel = librosa.power_to_db(mel)
        return log_mel
    
    def resize_spec(self, spec, target_shape=(128,256)): #resizing the CNN input to a fixed size(128x256)
        tensor = torch.tensor(spec).unsqueeze(0) #shape:(1,freq,time)
        if tensor.shape[-1] < target_shape[1]:
            pad = target_shape[1] - tensor.shape[-1]
            tensor = F.pad(tensor, (0, pad))
        else:
            tensor = tensor[:, :, :target_shape[1]]
        return tensor.numpy()
    
    def process_all(self):
        features = []
        for idx, (y, label) in enumerate(self.dataset):
            if len(y) < 256:  #padding short audio files
                y = np.pad(y, (0,256 - len(y)), mode= 'constant')

            mfcc = self.extract_mfcc(y)
            cqt = self.extract_cqt(y)
            lpc = self.extract_lpc(y)
            features.append({
                'mfcc' : mfcc,
                'lpc' : lpc,
                'cqt' : cqt,
                'label' : label
            })
            if idx % 100 == 0:
                print(f"Processed {idx} files")
        return features
    
    def process_for_cnn(self):
        spectrograms = []
        labels = []
        for idx, (y, label) in enumerate(self.dataset):
            if len(y) < 256 :
                y = np.pad(y, (0,256 - len(y)), mode = 'constant')
            
            log_mel = self.extract_log_mel(y)
            resized = self.resize_spec(log_mel)

            spectrograms.append(resized)
            labels.append(label)

            if idx % 100 == 0:
                print(f'Feed for CNN processes: {idx} files')
        
        return np.array(spectrograms), np.array(labels)


audio_folder  = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_train\flac"
protocol_file = r"C:\ASVSpoof19\LA\ASVspoof2019_LA_cm_protocols\ASVspoof2019.LA.cm.train.trn.txt"

# Run loader from OKR 1
loader = ASVspoof19Loader(audio_folder, protocol_file)
dataset = loader.load_dataset()

#runnign feature extraction
extractor = FeatureExtractor(dataset)
features = extractor.process_all()

#CNN feed processing
cnn_specs, cnn_labels = extractor.process_for_cnn()

#saving extracted features
np.save("features.npy", features)

#saving processed feed for CNN
np.save("cnn_spectrogram.npy", cnn_specs) #shape: (N,1,128,256)
np.save("cnn_labels.npy", cnn_labels)

#for KR4
print(f'saved {len(cnn_specs)} lpg mel spectrograms for CNN feed')
